In [16]:
# Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from scipy import stats 

In [17]:
# Preparing dataset from MusicBrainz
musicbrainz= pd.read_csv('musicbrainz_artist.txt', sep='|')
musicbrainz= musicbrainz.drop(columns=['ID'])
musicbrainz= musicbrainz[['Name','Gender','Genre','Type','Birthday']]
musicbrainz['Name']= musicbrainz['Name'].str.lower()
musicbrainz = musicbrainz[musicbrainz['Type'] == ' Person']

#Filtered by Electronic Music Genre 
genre_list = open('genrelist.txt','r')
lines = genre_list.readlines()
lines = lines[0]
genrelist = lines.split(',')
genrelist = list(set(genrelist))
genre_list.close()

musicbrainz= musicbrainz[musicbrainz['Genre'].str.contains('|'.join(genrelist))]
len(musicbrainz)

4083

In [18]:
# Preparing DataSet for Songkick all gigs
songkick= pd.read_csv('songkick_all_gigs_extended_version.tsv', sep='	')
songkick= songkick.drop(columns=['ID'])
songkick['Year'] = pd.DatetimeIndex(songkick['Date']).year
songkick= songkick.drop(columns=['Date','City','Venue'])
GigCount = songkick.groupby(['Name']).count()


GigCount = GigCount.drop(columns=['Country'])
GigCount = GigCount.rename(index=str, columns={'Year': 'GigCount'})

In [19]:
# Adding all countries for each artists
Countries = songkick.groupby('Name')['Country'].apply(','.join)
Countries = Countries.to_frame()

In [20]:
songkick['Year'] = songkick['Year'].apply(str)
ConcertYears = songkick.groupby('Name')['Year'].apply(','.join)
ConcertYears = ConcertYears.to_frame()

In [21]:
songkick = pd.merge(songkick, Countries, on='Name',how='inner')
#songkick.head()

In [22]:
songkick = pd.merge(songkick, GigCount, on='Name',how='inner')
#songkick.head()

In [23]:
songkick = pd.merge(songkick, ConcertYears, on='Name',how='inner')
#songkick.head()

In [24]:
songkick= songkick.drop(columns=['Country_x','Year_x'])  #'index' remove 
songkick= songkick.rename(index=str, columns={'Country_y': 'Countries', 'Year_y':'ConcertYears'})
songkick= songkick.drop_duplicates()
songkick = songkick.reset_index()
#songkick.head()

In [25]:
# Merging musicbrainz and songkick datasets on the basis of 'Name'

merge1= pd.merge(songkick, musicbrainz, on='Name',how='inner')
merge1= merge1.drop(columns=['index','Type','Genre'])
merge1.to_csv('Merge1.csv', index= True, sep= ',') #saved to merge1.csv
#merge1.head()

# giving the numbers of total gigs
# sum(merge1.GigCount)


In [26]:
# sum(merge1.GigCount)

In [27]:
# Preparing dataset for releases merged dataset
release= pd.read_csv('releases_merged.csv', sep=',')
release=release.drop(columns=['artist_id','release_id','label_name','Genre', 'Release_Name'])
release.rename(columns={'Year':'Release_Year'}, inplace=True)

release['Release_Year'] = release['Release_Year'].apply(str)
releaseYear = release.groupby('Name')['Release_Year'].apply(','.join)
releaseYear = releaseYear.to_frame()

AlbumCount = release.groupby(['Name']).count()
AlbumCount = AlbumCount.drop(columns=['Release_Year'])
AlbumCount = AlbumCount.rename(index=str, columns={'Price': 'AlbumCount'})

avgPrice = release.groupby('Name')['Price'].mean()
avgPrice = avgPrice.to_frame()

In [28]:
# Making new release dataset
release = pd.merge(release, releaseYear, on='Name',how='inner')
release = pd.merge(release, AlbumCount, on='Name',how='inner')
release = pd.merge(release, avgPrice, on='Name',how='inner')

# Dropping Price_x, Release_Year_x
release = release.drop(columns=['Price_x', 'Release_Year_x'])
release = release.drop_duplicates()
release = release.rename(index=str, columns={'Release_Year_y': 'Release_Years', 'Price_y':'AvgPrice'})

#release.head()

In [29]:
# Making Final Merged Dataset
finalMerge= pd.merge(merge1, release, on='Name', how='inner')
finalMerge.to_csv('FinalMerge.csv', index= True, sep= ',')

# Male-Female Separate Dataset
#male = finalMerge[finalMerge['Gender'] == ' Male']
#female = finalMerge[finalMerge['Gender'] == ' Female']

In [30]:
len(finalMerge)

549